# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function
    gfile = open(infile, "r")
    G = nx.DiGraph()
    num_nodes = dict()
    while True:
        line = gfile.readline().split()
        if(len(line)==0):
            break
        ind=line[0]
        ###skip comments
        if((ind.startswith('c')) |(ind.startswith('p'))) : continue
        ###stop once you hit double spaces
        if(ind.startswith('n')):
            G.add_node(line[1],demand=int(line[2]))
        if(ind.startswith('a')):
            node1 = line[1]
            node2 = line[2]
            if node1 not in G.nodes():
                G.add_node(node1,demand=0)
            if node2 not in G.nodes():
                G.add_node(node2,demand=0)
            edge = line[1],line[2]
            if(edge in num_nodes):
                intermediate_node = node1+node2+str(num_nodes[edge])
                G.add_node(intermediate_node,demand=0)
                G.add_edge(node1,intermediate_node,capacity=int(line[4]),weight=0)
                G.add_edge(intermediate_node,node2,capacity=int(line[4]),weight=int(line[5]))
                num_nodes[edge] = num_nodes[edge]+1 
            else:
                num_nodes[edge] =1
                G.add_edge(node1,node2,capacity=int(line[4]),weight=int(line[5]))
            
    return G

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [5]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    solver = pulp.LpProblem("min_cost",pulp.LpMinimize)
    check = dict()    
    ##creating vars and adding to dict
    for edge in G.edges():
        pulp_var = pulp.LpVariable(edge)
        check[edge]= pulp_var
    
    ##opj function
    solver += pulp.lpSum([check[i]*G[i[0]][i[1]]['weight'] for i in G.edges()])

    ##capacity and positive value constraints
    for edge in G.edges():
        solver += check[edge] <= G[edge[0]][edge[1]]['capacity']
        solver += check[edge] >= 0 
    
    ##demand constraints
    for node in G.nodes():
        in_edges  = G.in_edges(node)
        out_edges = G.out_edges(node)
        solver += pulp.lpSum([check[i] for i in in_edges]) - pulp.lpSum([check[i] for i in out_edges]) <= G.node[node]['demand']

    solver.solve()
    return pulp.value(solver.objective)

The following will check that the LP finds the same optimal values as previously.

In [6]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
